### 1. **Initialize Earth Engine & Import Libraries**

In [1]:
import ee
import folium
import numpy as np
import geemap
ee.Authenticate()
ee.Initialize(project='project-435712')

### 2. Define a Region of Interest

In [2]:
# Define the region of interest.
roi = ee.Geometry.Point([-122.26, 37.87])

### 3. **Load LANDSAT-8 DATA**

In [3]:
# Load the image collection.
img = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(roi) \
    .sort('CLOUD_COVER') \
    .first()

# Set the Visualization Parameters
visParamsT = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # Red, Green, Blue
    'min': 0,
    'max': 3000,
    'gamma': 1.2
}

In [4]:
# Import training data (Here I'm using Pre-trained Data)
training = ee.FeatureCollection('users/midekisa/Train_Cover_CA')

label = 'Class'
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
input = img.select(bands)

# Overlay the points on the imagery to get training data
trainImg = input.sampleRegions(**{
    'collection': training,
    'properties': [label],
    'scale': 30
})

trainingData = trainImg.randomColumn()
trainSet = trainingData.filter(ee.Filter.lt('random', 0.7))
testSet = trainingData.filter(ee.Filter.gte('random', 0.7))


### 4. **Classification Model**


In [5]:
# Create a random forest classifier
classifier = ee.Classifier.smileRandomForest(10).train(trainSet, label, bands)

# Classify the image
classified = input.classify(classifier)

# Define a palette for classification
landcoverPalette = [
    '#0c2c84',  # water (0)
    '#e31a1c',  # urban (1)
    '#005a32',  # forest(2)
    '#FF8000',  # crop  (3)
    '#969696',  # barren(4)
]

### 5. **FOLIUM App**

In [6]:
# Method to display ee imageTiles to Folium app
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add a drawing method
folium.Map.add_ee_layer = add_ee_layer

# Create folium map object
my_map = folium.Map(location=[38, -122], zoom_start=8)

# Add the layers to the map object
my_map.add_ee_layer(img, visParamsT, 'Landsat 2023')
my_map.add_ee_layer(classified, {'palette': landcoverPalette, 'min': 0, 'max': 4}, 'Classification')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
my_map

### 6. **Geemap**

In [7]:
# Load the image collection.
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(roi) \
    .sort('CLOUD_COVER')

# Applies scaling factors.
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Apply the scaling factors to the dataset.
dataset = dataset.map(apply_scale_factors)

# Select the first image from the dataset.
img = dataset.first()

In [8]:
# Set the visualization parameters.
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.2  # Adjust gamma if necessary
}

In [9]:
# Create a geemap map object
m = geemap.Map(center=[38, -122], zoom=8)
m.addLayer(img, visualization, 'Landsat 2023')
m.addLayer(classified, {'palette': landcoverPalette, 'min': 0, 'max': 4}, 'Classification')
m.addLayerControl()
m

Map(center=[38, -122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…